### Solutions for 02-Working_with_ASDF_Files

**Exercise 1:**

In [1]:
import asdf

**Read the file and use the `info` method to look at the contents**.

`info` has arguments which control the behavior. Use the defaults to look at the contents of the file.

In [2]:
af = asdf.open('../data/r0000101001001001001_0001_wfi01_cal.asdf')

In [3]:
af.info(max_rows=200)

root (AsdfObject)
├─asdf_library (Software)
│ ├─author (str): The ASDF Developers
│ ├─homepage (str): http://github.com/asdf-format/asdf
│ ├─name (str): asdf
│ └─version (str): 4.1.0
├─history (dict)
│ └─extensions (list)
│   ├─[0] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/core/extensions/core-1.6.0
│   │ ├─manifest_software (Software) ...
│   │ └─software (Software) ...
│   ├─[1] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/astronomy/gwcs/extensions/gwcs-1.4.0
│   │ ├─manifest_software (Software) ...
│   │ └─software (Software) ...
│   ├─[2] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://astropy.org/astropy/extensions/units-1.0.0
│   │ └─software (Software) ...
│   ├─[3] (ExtensionMetadata)
│   │ ├─ext

In [4]:
af.search('wcs')

root (AsdfObject)
└─roman (WfiImage) # Level 2 (L2) Calibrated Roman Wide Field Instrument (WFI) Rate Image.
  └─meta (dict)
    ├─cal_step (L2CalStep) # Level 2 Calibration Status
    │ └─assign_wcs (str): COMPLETE
    ├─wcs (WCS) # WCS object
    ├─wcs_fit_results (dict): {'<rot>': -2.6066827438249544e-05, '<scale>': 1.0, 'center': [-3.0895794266214 (truncated)
    └─wcsinfo (Wcsinfo): {'aperture_name': 'WFI01_FULL', 'v2_ref': 1312.9491452484797, 'v3_ref': -1040.78537 (truncated)

In [5]:
w = af['roman']['meta']['wcs']
print(w)

   From                   Transform                
---------- ----------------------------------------
  detector                            CompoundModel
      v2v3                           DVA_Correction
v2v3vacorr JWST tangent-plane linear correction. v1
  v2v3corr                                 v23tosky
     world                                     None


In [6]:
ra, dec = w(200, 300)
print(ra, dec)

269.97185510222323 65.98323704142405


In [7]:
sky = w.pixel_to_world(200, 300)
print(sky)

<SkyCoord (ICRS): (ra, dec) in deg
    (269.9718551, 65.98323704)>


In [8]:
af['roman']['data']

array([[ 1.6165170e-01,  8.1736194e-03,  2.1936220e-01, ...,
         2.0326480e-02,  2.3639268e-01,  1.6591458e-02],
       [ 2.3264721e-01,  1.2669931e-01,  2.5434250e-01, ...,
        -3.8930042e+03,  1.6126172e-01,  8.6484736e+03],
       [ 1.9691539e-01,  2.8878054e-01,  1.6772999e-01, ...,
         7.5255363e-04,  2.6196489e-01, -1.7857909e-03],
       ...,
       [ 2.0470999e-01,  1.8192567e-01,  2.3468970e-01, ...,
         2.1468423e-01,  1.7848723e-01,  2.0149684e-01],
       [ 1.6591151e-01,  1.6015893e-01,  2.5581256e-01, ...,
         2.1113476e-01,  2.0809896e-01,  1.4359643e-01],
       [ 1.9647996e-01,  1.8229012e-01,  2.0337157e-01, ...,
         2.4147762e-01,  2.1446401e-01,  1.7491595e-01]],
      shape=(4088, 4088), dtype=float32)

In [9]:
#af['roman']['data'][0, 0] = 999

**Exercise 2:**

Add `additionalProperties=false` to the schema and open the file

In [10]:
s = """
%YAML 1.1
---
$schema: http://stsci.edu/schemas/yaml-schema/draft-01

title: Mickey's pet
description: |
  Basic info and a picture of Mickie's 
  dog Pluto.

type: object
properties:
  age:
    title: The age of Pluto
    type: object
    properties:
      birthday:
        title: Pluto's first showing
        tag: tag:stsci.edu:asdf/time/time-1.1.0
  mass:
    title: How much he weighs.
    tag: tag:stsci.edu:asdf/unit/quantity-1.1.0
  picture:
    tag: tag:stsci.edu:asdf/core/ndarray-1.0.0
  name:
    title: Name
    type: string
required: [name, picture]
additionalProperties: false
...
"""

In [11]:
f = open('add-prop-1.0.0.yaml', mode='w')
f.write(s)
f.close()

In [12]:
# Expected validation error
afs = asdf.open('pluto.asdf', custom_schema='./add-prop-1.0.0.yaml')

ValidationError: Additional properties are not allowed ('asdf_library', 'birthday', 'history' were unexpected)

Failed validating 'additionalProperties' in schema:
    {'$schema': 'http://stsci.edu/schemas/yaml-schema/draft-01',
     'additionalProperties': False,
     'description': "Basic info and a picture of Mickie's \ndog Pluto.\n",
     'properties': {'age': {'properties': {'birthday': {'tag': 'tag:stsci.edu:asdf/time/time-1.1.0',
                                                        'title': "Pluto's "
                                                                 'first '
                                                                 'showing'}},
                            'title': 'The age of Pluto',
                            'type': 'object'},
                    'mass': {'tag': 'tag:stsci.edu:asdf/unit/quantity-1.1.0',
                             'title': 'How much he weighs.'},
                    'name': {'title': 'Name', 'type': 'string'},
                    'p

Take the original schema and add a new required property, called `friend`.

In [13]:
s = """
%YAML 1.1
---
$schema: http://stsci.edu/schemas/yaml-schema/draft-01

title: Mickey's pet
description: |
  Basic info and a picture of Mickie's 
  dog Pluto.

type: object
properties:
  age:
    title: The age of Pluto
    type: object
    properties:
      birthday:
        title: Pluto's first showing
        tag: tag:stsci.edu:asdf/time/time-1.1.0
  mass:
    title: How much he weighs.
    tag: tag:stsci.edu:asdf/unit/quantity-1.1.0
  picture:
    tag: tag:stsci.edu:asdf/core/ndarray-1.0.0
  name:
    title: Name
    type: string
  friend:
    type: string
    title: "Who is Pluto's friend?"
required: [name, picture, friend]
...
"""

In [18]:
f = open('pluto-friend-1.0.0.yaml', mode='w')
f.write(s)
f.close()

In [19]:
# Expected validation error
afs = asdf.open('pluto.asdf', custom_schema='./pluto-friend-1.0.0.yaml')

ValidationError: 'friend' is a required property

Failed validating 'required' in schema:
    {'$schema': 'http://stsci.edu/schemas/yaml-schema/draft-01',
     'description': "Basic info and a picture of Mickie's \ndog Pluto.\n",
     'properties': {'age': {'properties': {'birthday': {'tag': 'tag:stsci.edu:asdf/time/time-1.1.0',
                                                        'title': "Pluto's "
                                                                 'first '
                                                                 'showing'}},
                            'title': 'The age of Pluto',
                            'type': 'object'},
                    'friend': {'title': "Who is Pluto's friend?",
                               'type': 'string'},
                    'mass': {'tag': 'tag:stsci.edu:asdf/unit/quantity-1.1.0',
                             'title': 'How much he weighs.'},
                    'name': {'title': 'Name', 'type': 'string'},
                

In [20]:
asf = asdf.open('pluto.asdf', mode='rw')
asf['friend'] = 'Mickey'
#asf.update()
asf.write_to('pluto-friend.asdf')

In [21]:
asf = asdf.open('pluto-friend.asdf', custom_schema='./pluto-friend-1.0.0.yaml')